In [ ]:
#|default_exp models.InceptionTime

# InceptionTime

>An ensemble of deep Convolutional Neural Network (CNN) models, inspired by the Inception-v4 architecture

This is an unofficial PyTorch implementation created by Ignacio Oguiza (timeseriesAI@gmail.com) based on:

Fawaz, H. I., Lucas, B., Forestier, G., Pelletier, C., Schmidt, D. F., Weber, J. & Petitjean, F. (2019). 
<span style="color:dodgerblue">**InceptionTime: Finding AlexNet for Time Series Classification**</span>. arXiv preprint arXiv:1909.04939. 

Official InceptionTime tensorflow implementation: https://github.com/hfawaz/InceptionTime

In [6]:
#|export
from tsai.imports import *
from tsai.models.layers import *
from torchinfo import summary



In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# hyper parameter
num_classes = 30
batch_size = 64
learning_rage = 0.001
num_epochs = 10

In [2]:
#|export
# This is an unofficial PyTorch implementation by Ignacio Oguiza - timeseriesAI@gmail.com based on:

# Fawaz, H. I., Lucas, B., Forestier, G., Pelletier, C., Schmidt, D. F., Weber, J., ... & Petitjean, F. (2019). 
# InceptionTime: Finding AlexNet for Time Series Classification. arXiv preprint arXiv:1909.04939.
# Official InceptionTime tensorflow implementation: https://github.com/hfawaz/InceptionTime

class InceptionModule(Module):
    def __init__(self, ni, nf, ks=40, bottleneck=True):
        ks = [ks // (2**i) for i in range(3)]
        ks = [k if k % 2 != 0 else k - 1 for k in ks]  # ensure odd ks
        bottleneck = bottleneck if ni > 1 else False
        self.bottleneck = Conv1d(ni, nf, 1, bias=False) if bottleneck else noop
        self.convs = nn.ModuleList([Conv1d(nf if bottleneck else ni, nf, k, bias=False) for k in ks])
        self.maxconvpool = nn.Sequential(*[nn.MaxPool1d(3, stride=1, padding=1), Conv1d(ni, nf, 1, bias=False)])
        self.concat = Concat()
        self.bn = BN1d(nf * 4)
        self.act = nn.ReLU()

    def forward(self, x):
        input_tensor = x
        x = self.bottleneck(input_tensor)
        x = self.concat([l(x) for l in self.convs] + [self.maxconvpool(input_tensor)])
        return self.act(self.bn(x))


@delegates(InceptionModule.__init__)
class InceptionBlock(Module):
    def __init__(self, ni, nf=32, residual=True, depth=6, **kwargs):
        self.residual, self.depth = residual, depth
        self.inception, self.shortcut = nn.ModuleList(), nn.ModuleList()
        for d in range(depth):
            self.inception.append(InceptionModule(ni if d == 0 else nf * 4, nf, **kwargs))
            if self.residual and d % 3 == 2: 
                n_in, n_out = ni if d == 2 else nf * 4, nf * 4
                self.shortcut.append(BN1d(n_in) if n_in == n_out else ConvBlock(n_in, n_out, 1, act=None))
        self.add = Add()
        self.act = nn.ReLU()
        
    def forward(self, x):
        res = x
        for d, l in enumerate(range(self.depth)):
            x = self.inception[d](x)
            if self.residual and d % 3 == 2: res = x = self.act(self.add(x, self.shortcut[d//3](res)))
        return x

    
@delegates(InceptionModule.__init__)
class InceptionTime(Module):
    def __init__(self, c_in, c_out, seq_len=None, nf=32, nb_filters=None, **kwargs):
        nf = ifnone(nf, nb_filters) # for compatibility
        self.inceptionblock = InceptionBlock(c_in, nf, **kwargs)
        self.gap = GAP1d(1)
        self.fc = nn.Linear(nf * 4, c_out)

    def forward(self, x):
        x = self.inceptionblock(x)
        x = self.gap(x)
        x = self.fc(x)
        return x

In [3]:
from tsai.models.utils import count_parameters

In [12]:
bs = 16
vars = 1
seq_len = 12
c_out = 2
xb = torch.rand(bs, vars, seq_len)
test_eq(InceptionTime(vars,c_out)(xb).shape, [bs, c_out])
test_eq(InceptionTime(vars,c_out, bottleneck=False)(xb).shape, [bs, c_out])
test_eq(InceptionTime(vars,c_out, residual=False)(xb).shape, [bs, c_out])
test_eq(count_parameters(InceptionTime(3, 2)), 455490)

In [9]:
model = InceptionTime(3, 2)
summary(model)

Layer (type:depth-idx)                        Param #
InceptionTime                                 --
├─InceptionBlock: 1-1                         --
│    └─ModuleList: 2-1                        --
│    │    └─InceptionModule: 3-1              69,056
│    │    └─InceptionModule: 3-2              77,056
│    │    └─InceptionModule: 3-3              77,056
│    │    └─InceptionModule: 3-4              77,056
│    │    └─InceptionModule: 3-5              77,056
│    │    └─InceptionModule: 3-6              77,056
│    └─ModuleList: 2-2                        --
│    │    └─ConvBlock: 3-7                    640
│    │    └─BatchNorm1d: 3-8                  256
│    └─Add: 2-3                               --
│    └─ReLU: 2-4                              --
├─GAP1d: 1-2                                  --
│    └─AdaptiveAvgPool1d: 2-5                 --
│    └─Flatten: 2-6                           --
├─Linear: 1-3                                 258
Total params: 455,490
Trainable param

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataset, labels, root_dir, transform=None) -> None:
        # super().__init__()
        self.heartbeat = dataset
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
          idx = idx.tolist()
        
        # onehot_label = torch.eye(num_classes)[self.labels[idx] - 1] # one hot encordingは不要らしい　精度悪い場合試す必要あり
        one_hot = torch.nn.functional.one_hot(self.labels, num_classes=num_classes)
        return torch.tensor(self.heartbeat[idx]), onehot_label



    def __len__(self):
        return len(self.heartbeat)


dataset = MyDataset(df_data, df_labels, "./data/", transform=transforms.ToTensor())

In [ ]:
train_size = int(0.8 * len(df_data.values))
test_size = len(df_data.values) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
print(f"full: {len(dataset)} -> train: {len(train_set)}, test: {len(test_set)}")

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [ ]:
model = InceptionTime()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (signals, labels) in enumerate(train_loader):
    signals = torch.tensor(signals)
    signals = signals.float()
    signals = signals.to(device)
    labels = labels.to(device)

    # print(signals.size())
    outputs = model(signals)
    # print(outputs)
    loss = criterion(outputs, labels) # will check the shapes of outputs and labels

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # if (i + 1) % 15 == 0:
  print(f'Epoch [{epoch+1}/`{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  softmax = nn.Softmax()
  for i, (signals, one_hot_labels) in enumerate(test_loader):
    signals = torch.tensor(signals)
    signals = signals.float()
    signals = signals.to(device)
    one_hot_labels = one_hot_labels.to(device)
    # print(len(one_hot_labels))
    outputs = model(signals)
    for j, out in enumerate(outputs):
      outputs[j] = softmax(out)

    _, predicted = torch.max(outputs.data, 1) # predicted per batch size

    n_samples += one_hot_labels.size(0) # add batch_size
    # labels = [] # labels per batch size
    for k, labels in enumerate(one_hot_labels):
      # labels.append(torch.argmax(one_hot_labels[k]))
      if predicted[k] == torch.argmax(labels):
        # print(predicted[k], torch.argmax(labels[k]))
        n_correct += 1

    acc = 100.0 * n_correct / n_samples
    print(f'{n_correct} / {n_samples} = Acc: {acc} %')